<a href="https://colab.research.google.com/github/distinct93/Market-basket-Analysis/blob/main/data_collection_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

  import requests
from bs4 import BeautifulSoup
import pandas as pd

class WebScraper:
    def build_url(sort_by, page_number):
        sort_by='votes&page='
        base_url = "https://stackoverflow.com/questions?tab="
        return f'{base_url}{sort_by}{page_number}'

    def fetch_page(url):
        response = requests.get(url)
        return BeautifulSoup(response.content, "html.parser")

    def extract_questions(soup):
        classes_needed = ['.s-post-summary--stats-item-number', '.s-post-summary--content-title', '.s-post-summary--content-excerpt', '.post-tag']
        key_names = ['votes', 'summary', 'question', 'tags']
        question_classes = soup.select('.s-post-summary')
        datas = []

        for question in question_classes:
            question_data = {}
            for i, _class in enumerate(classes_needed):
                sub_element = question.select_one(_class)
                keyname = key_names[i]
                question_data[keyname] = sub_element.text if sub_element else None
            datas.append(question_data)

        return datas

    def scrape_stack_overflow(sort_by='votes&page=', start_page=1, end_page=50):
        all_data = []
        for page_number in range(start_page, end_page + 1):
            print(f"Scraping page {page_number}...")
            url = build_url(sort_by, page_number)
            soup = fetch_page(url)
            data = extract_questions(soup)
            all_data.extend(data)
        return pd.DataFrame(all_data)

if __name__ == "__main__":

    start_page = 1
    end_page = 10
    questions_df = scrape_stack_overflow(start_page=start_page, end_page=end_page)
    print(questions_df)